In [2]:
import numpy as np
import parmed as pmd

# MosDef packages
import foyer
from foyer import Forcefield
#import hoomd
import mbuild as mb



In [3]:
# User inputs

In [34]:
def compound(smiles_string):
    
    comp_mb = mb.load(smiles_string, smiles = True) # creates an mbuild compound instance
    comp_pmd = comp_mb.to_parmed() # convert the mbuild compound to a parmed compound
    comp_mass = np.sum([atom.mass for atom in comp_pmd.atoms]) # Mass of the compound
    comp_mb.mass = comp_mass # Update mb compound's mass

    GAFF = foyer.forcefields.load_GAFF()
    comp_typed = GAFF.apply(comp_mb, assert_dihedral_params=False)
    for atom_pmd, atom_mb in zip(comp_typed, comp_mb):
        atom_mb.name = "{}".format(atom_pmd.type)
    return comp_mb

def build_box(compound, num_compounds):
    
    #TO DO: Add, and use density parameter to determine box edge lengths
    
    box = mb.Box([10,10,10])
    system_box = mb.packing.fill_box(compound, num_compounds)
    system_box_pmd = system_box.to_parmed(box=box)
    system = GAFF.apply(system_box_pmd, assert_dihedral_params=False)
    
        # Why can't we start with an mbuild box of untyped compounds, then handle the typing here?
        # What is different about applying the ff to the box, vs creating a box with already typed molecules?
        # Where/How does Hoomd know which force field to use?
    
    return system

def sim():
    hoomd.context.initialize()
    pass

In [ ]:
from mbuild.formats.hoomd_simulation import create_hoomd_simulation
import hoomd
import hoomd.md
import hoomd.group

# No more context.initialize()??
create_hoomd_simulation(
    system, r_cut=1.2, auto_scale=True
)

_all = hoomd.group.all()
hoomd.md.integrate.mode_standard(dt=0.0001)
integrator = hoomd.md.integrate.nvt(group=_all, kT=1.0, tau=1)
hoomd.dump.gsd("start.gsd", period=None, group=_all, overwrite=True)
hoomd.dump.gsd("traj.gsd", period=1e5, group=_all, phase=0, overwrite=True)

hoomd.run(1e3)
hoomd.dump.gsd("out.gsd", period=None, group=_all, overwrite=True);